Code to run the rudimentary design(without CNN)

In [1]:
# Import all necessary files

# import numpy as np
# import torch
# import librosa
# import cv2
import os
import moviepy.editor as mp
from Components.Phoneme_Extraction import speech2text
from Components.p2fa_py3.p2fa import align
from Components.Viseme_Extraction import viseme
from Components.Mouth_Labelling import mouthLabelling

2023-03-05 05:41:41.705679: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 05:41:48.893094: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/rameez/Final_Proj/deepfake-detection/venv/lib/python3.8/site-packages/cv2/../../lib64:
2023-03-05 05:41:48.893126: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-05 05:42:01.976407: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic libra

In [8]:
                                            # Extract the audio from the input video file

# Insert Local Video File Path

video_path = "./Datasets/Video/obamaDeepfake.mp4"
clip = mp.VideoFileClip(video_path)
    
# Insert Local Audio File Path
audio_path = video_path.replace('.mp4', '.wav').replace('Video', 'Audio')
print(audio_path)
clip.audio.write_audiofile(audio_path)

OSError: MoviePy error: failed to read the duration of file ./Datasets/Video/obamaDeepfake.mp4.
Here are the file infos returned by ffmpeg:

ffmpeg version 4.2.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable

In [ ]:
# Converting Audio file to Text

transcription = speech2text.speech_transcription(audio_path)
print(transcription)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


IMAGINE THIS FOR A SECOND ONE MAN WITH TOTAL CONTROL OF BILLIONS OF PEOPLE STOLEN DATA ALL THEIR SECRETS THEIR LIVES THEIR FUTURES I OWE IT ALL TO SPECTRE SPECTRES SHOWED ME THAT WHOEVER CONTROLS THE DATA CONTROLS THE FUTURE AN


In [ ]:
# Write the variable value to the text file
text_path = video_path.replace('.mp4', '.txt').replace('Video', 'Text')

with open(text_path, 'w') as file:
    file.write(transcription.replace(' ', '\n'))

In [ ]:
# Phoneme Extraction

# Perform alignment using P2FA
phoneme_timestamps, word_timestamps, pff = align.align(audio_path, text_path)

# Extract the timestamps of a specific phoneme
target_phonemes = ["M", "B", "P"] # Closed mouth phonemes that we want to extract
for phoneme in target_phonemes:
    target_phoneme_timestamps = [(start, end) for p, start, end in phoneme_timestamps if p == phoneme]

# Print the timestamps of the target phoneme
print(f"Timestamps of '{phoneme}': {target_phoneme_timestamps}")

Resampling wav file from 44100 to 11025...
SKIPPING WORD IMAGINE
SKIPPING WORD THIS
SKIPPING WORD FOR
SKIPPING WORD A
SKIPPING WORD SECOND
SKIPPING WORD ONE
SKIPPING WORD MAN
SKIPPING WORD WITH
SKIPPING WORD TOTAL
SKIPPING WORD CONTROL
SKIPPING WORD OF
SKIPPING WORD BILLIONS
SKIPPING WORD OF
SKIPPING WORD PEOPLE
SKIPPING WORD STOLEN
SKIPPING WORD DATA
SKIPPING WORD ALL
SKIPPING WORD THEIR
SKIPPING WORD SECRETS
SKIPPING WORD THEIR
SKIPPING WORD LIVES
SKIPPING WORD THEIR
SKIPPING WORD FUTURES
SKIPPING WORD I
SKIPPING WORD OWE
SKIPPING WORD IT
SKIPPING WORD ALL
SKIPPING WORD TO
SKIPPING WORD SPECTRE
SKIPPING WORD SPECTRES
SKIPPING WORD SHOWED
SKIPPING WORD ME
SKIPPING WORD THAT
SKIPPING WORD WHOEVER
SKIPPING WORD CONTROLS
SKIPPING WORD THE
SKIPPING WORD DATA
SKIPPING WORD CONTROLS
SKIPPING WORD THE
SKIPPING WORD FUTURE
SKIPPING WORD AN


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Reube\\AppData\\Local\\Temp\\p2fa\\aligned.mlf'

In [ ]:
target_phoneme_timestamps = [(2.6564625850340136, 2.756235827664399), (3.045578231292517, 3.1054421768707483), (6.248299319727891, 6.318140589569161), (12.30453514739229, 12.414285714285713), (16.315419501133785, 16.345351473922904), (22.840589569160993, 22.89047619047619), (24.52675736961451, 24.576643990929703), (25.48458049886621, 25.544444444444444), (34.384353741496604, 34.42426303854876), (36.040589569161, 36.070521541950114), (41.168934240362816, 41.208843537414964), (43.363945578231295, 43.41383219954648), (44.6609977324263, 44.81065759637188), (46.436961451247164, 46.50680272108844), (48.39251700680273, 48.462358276644), (53.2015873015873, 53.31133786848073), (55.45646258503402, 55.5562358276644), (58.210204081632654, 58.24013605442177), (64.19659863945579, 64.30634920634921), (64.47596371882086, 64.55578231292517)]


In [ ]:
# Viseme Extraction

#import libraires
import cv2
import numpy as np
frames=viseme.extract_frames("./Datasets/Video/obamaDeepfake.mp4",target_phoneme_timestamps)

# Define the output folder
frame_path = "Datasets/Viseme_frames/"

# Loop through the frames and save them to the output folder
for i, (frame,index,frame_seqnum) in enumerate(frames):
    filename = "Frame_{}_{}.jpg".format(index,frame_seqnum)

    cv2.imwrite( frame_path + filename, frame)


Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Frame number out of range
Error: Fra

[mov,mp4,m4a,3gp,3g2,mj2 @ 0xb6ab140] moov atom not found


In [ ]:
# MAR Calculation

mar = mouthLabelling.mouth_aspect_ratio(frame_path)
print("Mouth aspect ratio: ", len(mar), " items")
print(len(mar))
for i in range(len(mar)):
  print (mar[i])

/home/rameez/Final_Proj/deepfake-detection
[INFO] loading facial landmark predictor...


RuntimeError: Error deserializing object of type int

In [ ]:
# Mismatch Calculation

MOUTH_AR_THRESH = 0.485

